In [1]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm
from selenium.webdriver.common.keys import Keys
import pandas as pd
import math
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [2]:
# 6호선 + 수집 안된 이미지
data = pd.read_csv("Final Res_for_6line.csv")
data = data[["상호명","상권업종중분류명","상권업종소분류명","도로명주소","역명","거리"]]
data["역"] = data["역명"].apply(lambda x : x.split(" ")[1])
data["역"] = data["역"].apply(lambda x : x.split("(")[0])
data["검색어"] = data.apply(lambda x : x["역"] + "역 " + str(x["상호명"]),axis=1)
data


,상호명,상권업종중분류명,상권업종소분류명,도로명주소,역명,거리,역,검색어
0,빈트리망원점,커피점/카페,커피전문점/카페/다방,서울특별시 마포구 망원로 22,6호선 마포구청,902.905260,마포구청,마포구청역 빈트리망원점
1,000간,커피점/카페,커피전문점/카페/다방,서울특별시 종로구 창신10가길 1-11,6호선 창신,519.957775,창신,창신역 000간
2,김밥천국,분식,라면김밥분식,서울특별시 성북구 종암로 87-6,6호선 월곡,760.370665,월곡,월곡역 김밥천국
3,클럽바리스타,커피점/카페,커피전문점/카페/다방,서울특별시 마포구 독막로 83,6호선 상수,31.738866,상수,상수역 클럽바리스타
4,훈고링고브레드,커피점/카페,커피전문점/카페/다방,서울특별시 마포구 잔다리로 130,6호선 망원,375.514951,망원,망원역 훈고링고브레드
...,...,...,...,...,...,...,...,...
1063,오리앙떼,커피점/카페,커피전문점/카페/다방,서울특별시 용산구 이태원로54길 16-9,6호선 한강진,273.699473,한강진,한강진역 오리앙떼
1064,써머레인,커피점/카페,커피전문점/카페/다방,서울특별시 용산구 이태원로55가길 49,6호선 한강진,425.285195,한강진,한강진역 써머레인
1065,라비앙코코,커피점/카페,커피전문점/카페/다방,서울특별시 용산구 녹사평대로40길 51,6호선 녹사평,173.097896,녹사평,녹사평역 라비앙코코
1066,올모스트홈카페,커피점/카페,커피전문점/카페/다방,서울특별시 용산구 회나무로13길 12,6호선 녹사평,631.676121,녹사평,녹사평역 올모스트홈카페


In [3]:

rating_df = pd.DataFrame(columns = ["UserID","ItemID","Rating","Timestamp"])
img_elem = pd.DataFrame(columns = ["ItemID","img_url"])
review_elem = pd.DataFrame(columns = ["ItemID", "review"])
user_review_id = {}
review_json = {}
image_json = {}


In [4]:
items = data["검색어"]
items

0       마포구청역 빈트리망원점
1           창신역 000간
2           월곡역 김밥천국
3         상수역 클럽바리스타
4        망원역 훈고링고브레드
            ...     
1063       한강진역 오리앙떼
1064       한강진역 써머레인
1065      녹사평역 라비앙코코
1066    녹사평역 올모스트홈카페
1067        상수역 지유명차
Name: 검색어, Length: 1068, dtype: object

In [ ]:
driver = webdriver.Chrome(r'C:/Users/user/Desktop/dd/Data_Handling/chromedriver.exe')
driver.maximize_window() 


count = 0
current = 0
goal = len(items)
for item in items :
    current += 1
    print('진행상황 : ', current,'/',goal,sep="")
    # 리뷰가 없을 때의 코드
    driver.get("https://map.kakao.com/") # 카카오 지도 접속하기
    searchbox = driver.find_element_by_xpath("//input[@id='search.keyword.query']") # 검색창에 입력하기
    searchbox.send_keys(item)
    time.sleep(2)
    searchbutton = driver.find_element_by_xpath("//button[@id='search.keyword.submit']") # 검색버튼 누르기
    driver.execute_script("arguments[0].click();", searchbutton)
    time.sleep(2)
    
    if len(driver.find_elements_by_xpath("//a[@class='moreview']")) != 0:
        print('식당 존재')
        driver.execute_script('window.open("about:blank", "_blank");') # 새 탭 열기
        reviewbutton = driver.find_element_by_xpath("//a[@class='numberofscore']")
        time.sleep(2)
        content_url = reviewbutton.get_attribute("href") 
        tabs = driver.window_handles
        driver.switch_to_window(tabs[1]) # 새 탭으로 이동
        driver.get(content_url) # 링크 열기
        time.sleep(3)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        review_lists = soup.select('.list_evaluation > li')
        print(len(review_lists))
        if len(review_lists) != 0 :
            for i, review in enumerate(review_lists) :
                user_review = review.select('.txt_comment > span') # 리뷰
                rating = review.select('.grade_star > em') # 별점
                try:
                    img_url = review_lists[i].select_one('a.link_photo > img ')['src']
                except:
                    continue
                user_id = review.select('.append_item > a[data-userid]') # user 정보 html 파싱
                timestamp = review.select(' div > span.time_write') #시간정보
                try:
                    row = {"UserID":user_id[0].get('data-userid'),"ItemID":item,"Rating":rating[0].text,"Timestamp":timestamp[0].text}
                    row = pd.DataFrame(row, index=[i])
                    rating_df = rating_df.append(row,ignore_index=True)
                    review_row = {"ItemID" : item, "review" : user_review[0].text}
                    review_row = pd.DataFrame(review_row, index=[i])
                    review_elem = review_elem.append(review_row, ignore_index = True)
                    try :
                        img_row = {"UserID":user_id[0].get('data-userid'),"ItemID" : item, "img_url" : img_url}
                        img_row = pd.DataFrame(img_row, index=[i])
                        img_elem = img_elem.append(img_row, ignore_index=True)
                    except :
                        img_row = {"UserID":None,"ItemID" : item, "img_url" : None}
                        img_row = pd.DataFrame(img_row, index=[i])
                        img_elem = img_elem.append(img_row, ignore_index=True)

                    time.sleep(3)
                except:
                    row = {"UserID":None,"ItemID":item,"Rating":None,"Timestamp":timestamp[0].text}
                    row = pd.DataFrame(row, index=[i])
                    rating_df = rating_df.append(row,ignore_index=True)
                    review_row = {"ItemID" : item, "review" : user_review[0].text}
                    review_row = pd.DataFrame(review_row, index=[i])
                    review_elem = review_elem.append(review_row, ignore_index = True)
                    try :
                        img_row = {"UserID":user_id[0].get('data-userid'),"ItemID" : item, "img_url" : img_url}
                        img_row = pd.DataFrame(img_row, index=[i])
                        img_elem = img_elem.append(img_row, ignore_index=True)
                    except :
                        img_row = {"UserID":user_id[0].get('data-userid'),"ItemID" : item, "img_url" : None}
                        img_row = pd.DataFrame(img_row, index=[i])
                        img_elem = img_elem.append(img_row, ignore_index=True)
                    time.sleep(1)

                
                    
        else :
            print("리뷰가 없습니다")
            
        try:
            for i in range(2,500):
                time.sleep(3)
                another_review = driver.find_element_by_xpath("//a[@data-page='" + str(i) + "']")
                another_review.click()
                time.sleep(3)
                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                review_lists = soup.select('.list_evaluation > li')
                if len(review_lists) != 0 :
                    for i, review in enumerate(review_lists) :
                        user_review = review.select('.txt_comment > span') # 리뷰
                        rating = review.select('.grade_star > em') # 별점
                        try:
                            img_url = review_lists[i].select_one('a.link_photo > img ')['src']
                        except:
                            continue
                        user_id = review.select('.append_item > a[data-userid]') # user 정보 html 파싱
                        timestamp = review.select(' div > span.time_write') #시간정보
                        try:
                            row = {"UserID":user_id[0].get('data-userid'),"ItemID":item,"Rating":rating[0].text,"Timestamp":timestamp[0].text}
                            row = pd.DataFrame(row, index=[i])
                            rating_df = rating_df.append(row,ignore_index=True)
                            review_row = {"UserID":user_id[0].get('data-userid'), "ItemID" : item, "review" : user_review[0].text}
                            review_row = pd.DataFrame(review_row, index=[i])
                            review_elem = review_elem.append(review_row, ignore_index = True)
                            try:
                                img_row = {"UserID":user_id[0].get('data-userid'),"ItemID" : item, "img_url" : img_url}
                                img_row = pd.DataFrame(img_row, index=[i])
                                img_elem = img_elem.append(img_row, ignore_index=True)
                            except:
                                img_row = {"UserID":user_id[0].get('data-userid'),"ItemID" : item, "img_url" : None}
                                img_row = pd.DataFrame(img_row, index=[i])
                                img_elem = img_elem.append(img_row, ignore_index=True)
                            time.sleep(1)
                            
                        except:
                            row = {"UserID":None,"ItemID":item,"Rating":None,"Timestamp":timestamp[0].text}
                            row = pd.DataFrame(row, index=[i])
                            rating_df = rating_df.append(row,ignore_index=True)
                            review_row = {"UserID":user_id[0].get('data-userid'),"ItemID" : item, "review" : user_review[0].text}
                            review_row = pd.DataFrame(review_row, index=[i])
                            review_elem = review_elem.append(review_row, ignore_index = True)
                            try :
                                img_row = {"UserID":user_id[0].get('data-userid'),"ItemID" : item, "img_url" : img_url}
                                img_row = pd.DataFrame(img_row, index=[i])
                                img_elem = img_elem.append(img_row, ignore_index=True)
                            except :
                                img_row = {"UserID":user_id[0].get('data-userid'),"ItemID" : item, "img_url" : None}
                                img_row = pd.DataFrame(img_row, index=[i])
                                img_elem = img_elem.append(img_row, ignore_index=True)
                            
                        
                            
        except:
            print("더 이상 리뷰 존재 X")
            driver.close()
        driver.switch_to_window(tabs[0])
        print("기본 페이지로 돌아가자")
            
    else:
        print("식당 존재 x")
        
        
        
